## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
# Load up store_income_data.csv

In [2]:
# Import libraries
import pandas as pd
import fuzzywuzzy 
from fuzzywuzzy import process, fuzz
import chardet 
from datetime import datetime

# Load the dataset
income_df = pd.read_csv('store_income_data_task.csv')

c:\Users\kirst\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
# Inspect unique values in the "country" column
unique_countries = income_df['country'].unique()
print("Unique values before cleaning:", unique_countries)

# Convert "country" column to lowercase and strip trailing whitespace
income_df['country'] = income_df['country'].str.lower().str.strip()

Unique values before cleaning: ['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. AfricaSouth Afric

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [4]:

# Manually map obvious matches
manual_mapping = {
    'uk': 'united kingdom',
    'u.k': 'united kingdom',
    'u.k.': 'united kingdom',
    'u.k/': 'united kingdom',
    'england': 'united kingdom',
    'england.': 'united kingdom',
    'england/': 'united kingdom',
    'britain': 'united kingdom',
    'britain.': 'united kingdom',
    'britain/': 'united kingdom',
    'america': 'united states',
    'america.': 'united states',
    'america/': 'united states',
    'united states of america': 'united states',
    'united states of america.': 'united states',
    'united states of america/': 'united states',
    'sa': 'south africa',
    's.a.': 'south africa',
    's.a./': 'south africa',
    's.a..': 'south africa',
    's. africasouth africa': 'south africa',
    's. africasouth africa/': 'south africa',
    's. africasouth africa.': 'south africa',
    'uk.': 'united kingdom',
    'uk/': 'united kingdom',
    'sa.': 'south africa',
    'sa/': 'south africa',
    '.': 'unknown',
    '/': 'unknown',
    '': 'unknown'
}

# Apply manual mapping
income_df['country'] = income_df['country'].replace(manual_mapping)

# Function to replace rows in the provided column of the 
# provided DataFrame
# that match the provided string above the provided ratio with 
# the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio=90):
    # Get a list of unique strings
    strings = df[column].unique()
    
    # Get the closest matches to our input string
    matches = process.extract(string_to_match, strings, 
                              limit=len(strings), scorer=fuzz.token_sort_ratio)

    # Only get matches with a ratio above the min_ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]

    # Get the rows of all the close matches in the DataFrame
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input string
    df.loc[rows_with_matches, column] = string_to_match

# Apply fuzzy matching to standardize country names
for standard_country in ['united kingdom', 'united states', 'south africa']:
    replace_matches_in_column(income_df, 'country', standard_country)

# Handle remaining NaN or empty string values
income_df['country'] = income_df['country'].fillna('unknown')
income_df['country'] = income_df['country'].str.title()

# Check the final unique values
cleaned_unique_countries = income_df['country'].unique()
print(f"Unique values after cleaning: {cleaned_unique_countries}")

# Save the cleaned data (optional)
income_df.to_csv('cleaned_data.csv', index=False)

Unique values after cleaning: ['United States' 'United Kingdom' 'South Africa' 'Unknown']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [5]:
# Convert 'date_measured' column to datetime format
income_df['date_measured'] = pd.to_datetime(income_df['date_measured'],
                                            format='%d-%m-%Y', errors='coerce')

# Get the current date
current_date = datetime.today()

# Calculate the number of days ago for each date and add it as a new column
income_df['days_ago'] = (current_date - income_df['date_measured']).dt.days

# Inspect the DataFrame to see the new 'days_ago' column
print(income_df[['date_measured', 'days_ago']].head())

  date_measured  days_ago
0    2006-02-04      6762
1    2006-01-04      6793
2    2003-09-12      7638
3    2006-05-08      6669
4    1973-01-21     18829
